# API Face Samples

## Objective

The objective of this script is to automate the process of creating a person for each image, adding the largest face from the image to that person, and then using the person ID against an array of created person IDs for face identification. It iterates through a folder of images to perform these tasks and uses a test image to identify the enrolled faces.


In [ ]:
import os
from shared_functions import create_person, add_person_face, identify_faces, create_dynamic_person_group

# Replace with your Azure Face API subscription key and endpoint
FACE_KEY = os.environ["FACE_API_KEY"]
FACE_ENDPOINT = os.environ["FACE_ENDPOINT_URL"]
folder_path = 'path_to_folder_containing_images'
test_image_path = 'path_to_test_image.jpg'
dynamic_person_group_id = "dynamic_person_group_id"

persond_ids = []
# Iterate through all images in the folder
for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(folder_path, filename)
        person_name = os.path.splitext(filename)[0]
        # For each image, create a person and add a face to that person
        person_id = create_person(FACE_KEY, FACE_ENDPOINT, person_name, "sample=enroll_single_face_per_person")
        if person_id:
            persond_ids.append(person_id)
        persisted_face_id = add_person_face(FACE_KEY, FACE_ENDPOINT, image_path, person_id, "sample=enroll_single_face_per_person")
        print(f"Added face from {filename} with persisted_face_id: {persisted_face_id} to person_id: {person_id}")

print(f"Person IDs: {persond_ids}")
# Identify faces in the test image using the person IDs
identify_faces(FACE_KEY, FACE_ENDPOINT, test_image_path, person_ids=persond_ids, injection_header="sample=enroll_single_face_per_person")

# Create a dynamic person group based on the person IDs
if create_dynamic_person_group(FACE_KEY, FACE_ENDPOINT, dynamic_person_group_id, persond_ids, "sample=enroll_single_face_per_person"):
    print(f"Created dynamic person group with ID: {dynamic_person_group_id}")

    # Identify faces in the test image using the dynamic person group"""
    identify_faces(FACE_KEY, FACE_ENDPOINT, test_image_path, dynamic_person_group_id=dynamic_person_group_id, injection_header="sample=enroll_single_face_per_person")